In [1]:
from dotenv import dotenv_values
import os
from pathlib import Path
import traceback
from tqdm.notebook import tqdm  # progress bar for notebooks
import pandas as pd

env_path = "/Users/crissafrancisco/advanced-track.ie/.env"


config = dotenv_values(env_path)
print("config dict:", config)

os.environ.update(config)

print("DB_USERNAME:", os.getenv("DB_USERNAME"))
print("DB_HOST:", os.getenv("DB_HOST"))

config dict: OrderedDict({'DB_USERNAME': 'daniel', 'DB_PASSWORD': 'db2dani', 'DB_HOST': '52.211.123.34', 'DB_PORT': '25010', 'DB_NAME': 'IEMASTER'})
DB_USERNAME: daniel
DB_HOST: 52.211.123.34


In [2]:
import sys
import os

# Add project root to path
project_root = os.path.abspath("..")
sys.path.append(project_root)


In [3]:
# Importing Class
from src.connector import DB2Connector

# Connecting to the Database
db = DB2Connector()

# Listing Schemas
db.list_schemas()

,SCHEMA
0,DB2ADMIN
1,IEPLANE
2,LOCALADMIN
3,NULLID
4,SQLJ
5,SYSCAT
6,SYSFUN
7,SYSIBM
8,SYSIBMADM
9,SYSIBMINTERNAL


In [4]:
db.list_tables(schema = 'IEPLANE')


,tabschema,tabname,TYPE
0,IEPLANE,AIRPLANES,T
1,IEPLANE,AIRPORTS,T
2,IEPLANE,COUNTRIES,T
3,IEPLANE,DEPARTMENT,T
4,IEPLANE,EMPLOYEE,T
5,IEPLANE,FLIGHTS,T
6,IEPLANE,PASSENGERS,T
7,IEPLANE,ROUTES,T
8,IEPLANE,TICKETS,T


In [5]:
df = db.read_sql("""
    SELECT *
    FROM IEPLANE.FLIGHTS
    FETCH FIRST 10 ROWS ONLY
""")

df.head()

,flight_id,flight_leg,frequency,route_code,departure,arrival,airplane,price_economy,price_premium,price_business
0,IE0006,1,F1,R002,2000-01-03 10:20:00,2000-01-03 11:26:00,IE33216,97.13,128.74,159.87
1,IE0006,2,F1,R003,2000-01-03 12:26:00,2000-01-03 14:13:00,IE33216,164.01,240.45,293.55
2,IE0006,0,F1,R001,2000-01-03 07:05:00,2000-01-03 09:20:00,IE33216,229.82,300.74,333.37
3,IE0002,0,F1,R017,2000-01-03 06:45:00,2000-01-03 07:39:00,IE10328,70.07,77.29,118.45
4,IE0007,0,F1,R020,2000-01-03 05:50:00,2000-01-03 07:57:00,IE59049,198.22,268.77,341.17


In [4]:
pd.read_sql("SELECT COUNT(*) FROM IEPLANE.TICKETS", engine)

,1
0,35383337


In [5]:
pd.read_sql("SELECT COUNT(*) FROM IEPLANE.FLIGHTS", engine)


,1
0,297471


In [6]:
pd.read_sql("SELECT COUNT(*) FROM IEPLANE.AIRPLANES", engine)

,1
0,120


In [7]:
pd.read_sql(
    "SELECT SUM(total_amount) AS total_revenue FROM IEPLANE.TICKETS",
    engine
)


,total_revenue
0,2.835791e+10


In [8]:
# cheap and safe: returns just 1 row with all column names in the DataFrame
pd.read_sql("SELECT * FROM IEPLANE.TICKETS FETCH FIRST 1 ROW ONLY", engine).columns.tolist()


['ticket_id',
 'passenger_id',
 'flight_id',
 'route_code',
 'departure',
 'class',
 'seat',
 'price',
 'airport_tax',
 'local_tax',
 'total_amount']

In [9]:
# Revenue per ticket class
import pandas as pd

sql_revenue_by_class = """
SELECT t.class        AS ticket_class,
       COUNT(*)       AS tickets_sold,
       SUM(t.price)   AS base_revenue,
       SUM(t.airport_tax) AS ancillary_revenue,
       SUM(t.total_amount) AS total_revenue
FROM IEPLANE.TICKETS t
GROUP BY t.class
ORDER BY SUM(t.total_amount) DESC
"""

revenue_by_class = pd.read_sql(sql_revenue_by_class, engine)
revenue_by_class


,ticket_class,tickets_sold,base_revenue,ancillary_revenue,total_revenue
0,E,22251064,1.664808e+10,2.728272e+08,2.041690e+10
1,B,12447496,5.619399e+09,1.563909e+08,6.955802e+09
2,P,684777,8.087475e+08,6.630144e+06,9.852112e+08


In [10]:
# Revenue share
revenue_by_class["revenue_share_pct"] = (
    revenue_by_class["total_revenue"] /
    revenue_by_class["total_revenue"].sum()
) * 100

revenue_by_class


,ticket_class,tickets_sold,base_revenue,ancillary_revenue,total_revenue,revenue_share_pct
0,E,22251064,1.664808e+10,2.728272e+08,2.041690e+10,71.997184
1,B,12447496,5.619399e+09,1.563909e+08,6.955802e+09,24.528613
2,P,684777,8.087475e+08,6.630144e+06,9.852112e+08,3.474203


In [11]:
# Revenue share donut chart
import plotly.express as px

fig = px.pie(
    revenue_by_class,
    names="ticket_class",
    values="total_revenue",
    hole=0.6,
    title="Revenue Share by Ticket Class"
)

fig.update_traces(textposition='inside', textinfo='percent+label')

fig.show()


In [12]:
# Ancillary revenue
total_ancillary = revenue_by_class["ancillary_revenue"].sum()
total_revenue = revenue_by_class["total_revenue"].sum()

ancillary_pct = (total_ancillary / total_revenue) * 100
ancillary_pct


np.float64(1.5369547820251106)

In [13]:
# Top 10 routes byr evenue
sql_top_routes = """
SELECT route_code,
       SUM(total_amount) AS route_revenue,
       COUNT(*) AS tickets_sold
FROM IEPLANE.TICKETS
GROUP BY route_code
ORDER BY route_revenue DESC
FETCH FIRST 10 ROWS ONLY
"""

top_routes = pd.read_sql(sql_top_routes, engine)
top_routes


,route_code,route_revenue,tickets_sold
0,R057,1.425896e+09,899893
1,R056,1.416713e+09,894582
2,R034,1.238314e+09,933411
3,R035,1.236002e+09,932052
4,R061,1.202351e+09,919021
5,R030,1.200054e+09,935350
6,R060,1.189105e+09,912846
7,R031,1.187331e+09,924291
8,R044,9.237370e+08,401791
9,R045,9.213942e+08,400178


In [14]:
# Visualize top routes
import plotly.express as px

fig = px.bar(
    top_routes,
    x="route_revenue",
    y="route_code",
    orientation="h",
    title="Top 10 Routes by Revenue",
    text_auto=True
)

fig.update_layout(
    xaxis_title="Revenue",
    yaxis_title="Route Code",
    yaxis=dict(autorange="reversed")  # highest at top
)

fig.show()


In [15]:
import pandas as pd

airplanes = pd.read_sql("SELECT * FROM IEPLANE.AIRPLANES FETCH FIRST 5 ROWS ONLY", engine)
routes = pd.read_sql("SELECT * FROM IEPLANE.ROUTES FETCH FIRST 5 ROWS ONLY", engine)

print("AIRPLANES columns:", airplanes.columns.tolist())
print("ROUTES columns:", routes.columns.tolist())


AIRPLANES columns: ['aircraft_registration', 'model', 'seats_business', 'seats_premium', 'seats_economy', 'crew_members', 'build_date', 'fuel_gallons_hour', 'maintenance_last_acheck', 'maintenance_last_bcheck', 'maintenance_takeoffs', 'maintenance_flight_hours', 'total_flight_distance']
ROUTES columns: ['route_code', 'origin', 'destination', 'parent_route', 'leg_number', 'distance', 'flight_minutes']


In [16]:
# Revenue Available per Seat Mile (RASM) Overall
sql_rasm_fast = """
WITH ticket_rev_by_flight AS (
    SELECT flight_id,
           DECIMAL(SUM(DECIMAL(total_amount, 31, 2)), 31, 2) AS flight_revenue
    FROM IEPLANE.TICKETS
    GROUP BY flight_id
),
flight_asm AS (
    SELECT f.flight_id,
           DECIMAL((a.seats_economy + a.seats_premium + a.seats_business), 31, 0) AS seats,
           DECIMAL(r.distance, 31, 2) AS distance
    FROM IEPLANE.FLIGHTS f
    JOIN IEPLANE.AIRPLANES a
         ON f.airplane = a.aircraft_registration
    JOIN IEPLANE.ROUTES r
         ON f.route_code = r.route_code
)
SELECT
    DECIMAL(SUM(tr.flight_revenue), 31, 2) AS total_revenue,
    DECIMAL(SUM(fa.seats * fa.distance), 31, 2) AS total_available_seat_miles,
    DECIMAL(SUM(tr.flight_revenue), 31, 8) / NULLIF(DECIMAL(SUM(fa.seats * fa.distance), 31, 8), 0) AS rasm
FROM ticket_rev_by_flight tr
JOIN flight_asm fa
     ON tr.flight_id = fa.flight_id
"""

rasm_df = pd.read_sql(sql_rasm_fast, engine)
rasm_df



,total_revenue,total_available_seat_miles,rasm
0,2.934002e+14,1.708670e+10,17171.0


In [17]:
# RASM by Route (Top 20)
sql_rasm_route_fast = """
WITH ticket_rev_by_flight AS (
    SELECT flight_id,
           DECIMAL(SUM(DECIMAL(total_amount, 31, 2)), 31, 2) AS flight_revenue
    FROM IEPLANE.TICKETS
    GROUP BY flight_id
),
flight_dim AS (
    SELECT f.flight_id,
           f.route_code,
           DECIMAL((a.seats_economy + a.seats_premium + a.seats_business), 31, 0) AS seats,
           DECIMAL(r.distance, 31, 2) AS distance
    FROM IEPLANE.FLIGHTS f
    JOIN IEPLANE.AIRPLANES a
         ON f.airplane = a.aircraft_registration
    JOIN IEPLANE.ROUTES r
         ON f.route_code = r.route_code
)
SELECT
    d.route_code,
    DECIMAL(SUM(tr.flight_revenue), 31, 2) AS total_revenue,
    DECIMAL(SUM(d.seats * d.distance), 31, 2) AS asm,
    DECIMAL(SUM(tr.flight_revenue), 31, 8) / NULLIF(DECIMAL(SUM(d.seats * d.distance), 31, 8), 0) AS rasm
FROM ticket_rev_by_flight tr
JOIN flight_dim d
     ON tr.flight_id = d.flight_id
GROUP BY d.route_code
ORDER BY rasm DESC
FETCH FIRST 20 ROWS ONLY
"""

rasm_route_df = pd.read_sql(sql_rasm_route_fast, engine)
rasm_route_df


,route_code,total_revenue,asm,rasm
0,R001,6.144557e+12,None,None
1,R020,4.299672e+12,None,None
2,R019,1.576089e+12,None,None
3,R018,1.576089e+12,None,None
4,R017,1.576089e+12,None,None
5,R016,1.419351e+12,None,None
6,R015,1.419351e+12,None,None
7,R014,1.419351e+12,None,None
8,R013,2.289353e+12,None,None
9,R012,2.289353e+12,None,None


In [18]:
pd.read_sql(
    "SELECT * FROM IEPLANE.FLIGHTS FETCH FIRST 5 ROWS ONLY",
    engine
)


,flight_id,flight_leg,frequency,route_code,departure,arrival,airplane,price_economy,price_premium,price_business
0,IE0006,1,F1,R002,2000-01-03 10:20:00,2000-01-03 11:26:00,IE33216,97.13,128.74,159.87
1,IE0006,2,F1,R003,2000-01-03 12:26:00,2000-01-03 14:13:00,IE33216,164.01,240.45,293.55
2,IE0006,0,F1,R001,2000-01-03 07:05:00,2000-01-03 09:20:00,IE33216,229.82,300.74,333.37
3,IE0002,0,F1,R017,2000-01-03 06:45:00,2000-01-03 07:39:00,IE10328,70.07,77.29,118.45
4,IE0007,0,F1,R020,2000-01-03 05:50:00,2000-01-03 07:57:00,IE59049,198.22,268.77,341.17


In [19]:
sql_rasm_fixed = """
WITH ticket_rev_by_flight AS (
    SELECT flight_id,
           DECIMAL(SUM(DECIMAL(total_amount, 31, 2)), 31, 2) AS flight_revenue
    FROM IEPLANE.TICKETS
    GROUP BY flight_id
),
flight_dim AS (
    SELECT 
        f.flight_id,
        DECIMAL((a.seats_economy + a.seats_premium + a.seats_business), 31, 0) AS seats,
        DECIMAL(r.distance, 31, 2) AS distance
    FROM IEPLANE.FLIGHTS f
    JOIN IEPLANE.AIRPLANES a
         ON f.airplane = a.aircraft_registration
    JOIN IEPLANE.ROUTES r
         ON f.route_code = r.route_code
        AND f.flight_leg = r.leg_number
)
SELECT
    DECIMAL(SUM(tr.flight_revenue), 31, 2) AS total_revenue,
    DECIMAL(SUM(d.seats * d.distance), 31, 2) AS total_available_seat_miles,
    DECIMAL(SUM(tr.flight_revenue), 31, 8) 
      / NULLIF(DECIMAL(SUM(d.seats * d.distance), 31, 8), 0) AS rasm
FROM ticket_rev_by_flight tr
JOIN flight_dim d
  ON tr.flight_id = d.flight_id
"""

rasm_df = pd.read_sql(sql_rasm_fixed, engine)
rasm_df


,total_revenue,total_available_seat_miles,rasm
0,2.934002e+14,1.708670e+10,17171.0


In [20]:
# Revenue per available seat mile (RASM)

sql_rasm = """
SELECT 
    SUM(t.total_amount) AS total_revenue,
    SUM(a.capacity * r.distance) AS total_available_seat_miles,
    SUM(t.total_amount) / SUM(a.capacity * r.distance) AS rasm
FROM IEPLANE.TICKETS t
JOIN IEPLANE.FLIGHTS f 
     ON t.flight_id = f.flight_id
JOIN IEPLANE.AIRPLANES a 
     ON f.airplane = a.airplane_id
JOIN IEPLANE.ROUTES r 
     ON f.route_code = r.route_code
"""

rasm_df = pd.read_sql(sql_rasm, engine)
rasm_df



DatabaseError: Execution failed on sql '
SELECT 
    SUM(t.total_amount) AS total_revenue,
    SUM(a.capacity * r.distance) AS total_available_seat_miles,
    SUM(t.total_amount) / SUM(a.capacity * r.distance) AS rasm
FROM IEPLANE.TICKETS t
JOIN IEPLANE.FLIGHTS f 
     ON t.flight_id = f.flight_id
JOIN IEPLANE.AIRPLANES a 
     ON f.airplane = a.airplane_id
JOIN IEPLANE.ROUTES r 
     ON f.route_code = r.route_code
': (ibm_db_dbi.ProgrammingError) ibm_db_dbi::ProgrammingError: Statement Execute Failed: [IBM][CLI Driver][DB2/LINUXX8664] SQL0206N  "A.AIRPLANE_ID" is not valid in the context where it is used.  SQLSTATE=42703 SQLCODE=-206
[SQL: 
SELECT 
    SUM(t.total_amount) AS total_revenue,
    SUM(a.capacity * r.distance) AS total_available_seat_miles,
    SUM(t.total_amount) / SUM(a.capacity * r.distance) AS rasm
FROM IEPLANE.TICKETS t
JOIN IEPLANE.FLIGHTS f 
     ON t.flight_id = f.flight_id
JOIN IEPLANE.AIRPLANES a 
     ON f.airplane = a.airplane_id
JOIN IEPLANE.ROUTES r 
     ON 

### Saving Datasets as Parquets

In [5]:
# Config
SCHEMA = "IEPLANE"
TABLES = [
    "AIRPLANES",
    "AIRPORTS",
    "COUNTRIES",
    "DEPARTMENT",
    "EMPLOYEE",
    "FLIGHTS",
    "PASSENGERS",
    "ROUTES",
    "TICKETS",
]
OUTPUT_DIR = Path("datasets")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

LIMIT_PER_TABLE = 10_000  # change if you want fewer/more
CHUNKED = False           # set True to use chunked mode (safer for low RAM)
CHUNK_SIZE = 2000         # chunk size when CHUNKED=True

In [6]:
manifest = []  # will hold info about parts produced

for table in tqdm(TABLES, desc="Tables (chunked)"):
    base_dir = OUTPUT_DIR / table.lower()
    base_dir.mkdir(parents=True, exist_ok=True)
    print(f"\nChunked exporting {SCHEMA}.{table} -> {base_dir} (limit={LIMIT_PER_TABLE}, chunk_size={CHUNK_SIZE})")

    rows_written = 0
    part_i = 0
    try:
        # Using connector.read_sql with a fully qualified query avoids bind issues
        query = f"SELECT * FROM {SCHEMA}.{table} FETCH FIRST {LIMIT_PER_TABLE} ROWS ONLY"
        chunks = db.read_sql(query, params=None, chunksize=CHUNK_SIZE)

        part_files = []
        for chunk in chunks:
            if chunk is None or chunk.empty:
                break
            part_path = base_dir / f"{table.lower()}_part{part_i:03d}.parquet"
            chunk.to_parquet(part_path.as_posix(), index=False)
            part_files.append(str(part_path))
            rows_written += len(chunk)
            print(f" - wrote part {part_i:03d} ({len(chunk)} rows) -> {part_path.name}")
            part_i += 1
            if rows_written >= LIMIT_PER_TABLE:
                break

        status = "ok" if rows_written > 0 else "empty"
        manifest.append({"table": table, "rows": rows_written, "parts": part_files, "status": status})

    except Exception as e:
        print(f" - ERROR (chunked) exporting {table}: {e}")
        traceback.print_exc()
        manifest.append({"table": table, "rows": None, "parts": [], "status": f"error: {e}"})

# present manifest nicely
manifest_df = pd.DataFrame(manifest)
manifest_df

Tables (chunked):   0%|          | 0/9 [00:00<?, ?it/s]


Chunked exporting IEPLANE.AIRPLANES -> datasets/airplanes (limit=10000, chunk_size=2000)
 - wrote part 000 (120 rows) -> airplanes_part000.parquet

Chunked exporting IEPLANE.AIRPORTS -> datasets/airports (limit=10000, chunk_size=2000)
 - wrote part 000 (30 rows) -> airports_part000.parquet

Chunked exporting IEPLANE.COUNTRIES -> datasets/countries (limit=10000, chunk_size=2000)
 - wrote part 000 (196 rows) -> countries_part000.parquet

Chunked exporting IEPLANE.DEPARTMENT -> datasets/department (limit=10000, chunk_size=2000)
 - wrote part 000 (22 rows) -> department_part000.parquet

Chunked exporting IEPLANE.EMPLOYEE -> datasets/employee (limit=10000, chunk_size=2000)
 - wrote part 000 (1598 rows) -> employee_part000.parquet

Chunked exporting IEPLANE.FLIGHTS -> datasets/flights (limit=10000, chunk_size=2000)
 - wrote part 000 (2000 rows) -> flights_part000.parquet
 - wrote part 001 (2000 rows) -> flights_part001.parquet
 - wrote part 002 (2000 rows) -> flights_part002.parquet
 - wrot

,table,rows,parts,status
0,AIRPLANES,120,[datasets/airplanes/airplanes_part000.parquet],ok
1,AIRPORTS,30,[datasets/airports/airports_part000.parquet],ok
2,COUNTRIES,196,[datasets/countries/countries_part000.parquet],ok
3,DEPARTMENT,22,[datasets/department/department_part000.parquet],ok
4,EMPLOYEE,1598,[datasets/employee/employee_part000.parquet],ok
5,FLIGHTS,10000,"[datasets/flights/flights_part000.parquet, dat...",ok
6,PASSENGERS,10000,[datasets/passengers/passengers_part000.parque...,ok
7,ROUTES,59,[datasets/routes/routes_part000.parquet],ok
8,TICKETS,10000,"[datasets/tickets/tickets_part000.parquet, dat...",ok


In [7]:
manifest_csv = OUTPUT_DIR / "export_manifest.csv"
manifest_df.to_csv(manifest_csv, index=False)
print("Saved manifest to", manifest_csv)

Saved manifest to datasets/export_manifest.csv
